In [2]:
import pandas as pd
from angle_emb import AnglE
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("data/data-with-media/steam.csv")
df.sample(3)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
15136,654940,RXE,2017-07-21,1,Buce Studios LLC,Buce Studios LLC,windows;mac;linux,0,Single-player;Multi-player;Steam Achievements;...,Indie;Simulation;Early Access,Early Access;Indie;Simulation,2,1,3,0,0,0-20000,4.79
12395,566980,Crashimals,2017-09-07,1,Rogue Earth LLC,GAMEPUMP,windows,0,Single-player,Action;Casual;Indie;Strategy,Action;Casual;Indie,0,10,0,0,0,0-20000,0.79
18844,757690,Limit of defense,2017-12-16,1,Jl Apps,Jl Apps,windows,0,Single-player,Indie,Indie,0,0,1,0,0,0-20000,2.09


In [4]:
media_df = pd.read_csv("data/data-with-media/steam_media_data.csv")
media_df.sample(3)

,steam_appid,header_image,screenshots,background,movies
147,6000,https://steamcdn-a.akamaihd.net/steam/apps/600...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/600...,"[{'id': 256668530, 'name': 'Republic Commando ..."
7830,434160,https://steamcdn-a.akamaihd.net/steam/apps/434...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/434...,"[{'id': 256685194, 'name': 'A Hole New World -..."
21042,816750,https://steamcdn-a.akamaihd.net/steam/apps/816...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/816...,"[{'id': 256710161, 'name': 'Travildorn', 'thum..."


In [5]:
games_df = pd.read_csv("data/data-with-media/steam.csv")
media_df = pd.read_csv("data/data-with-media/steam_media_data.csv")
description_df = pd.read_csv("data/data-with-media/steam_description_data.csv")


In [6]:
df = games_df.merge(media_df, left_on="appid" , right_on="steam_appid").merge(description_df, on="steam_appid")
game_texts = []
for _, row in df.iterrows():
    game_text = f'''Game name : {row["name"]}
Developer : {row["developer"]}
Genres : {row["genres"]}
Summary : {row["short_description"]}'''
    game_texts.append(game_text)
df["text"] = game_texts
df["minimum_owners"] = df ["owners"].apply(lambda v : int(v.split("-")[0]))
df.sample(3).T

,4542,19813,143
appid,341780,785870,4880
name,Chronicles of a Dark Lord: Episode II War of T...,Admine,Cossacks: European Wars
release_date,2015-01-16,2018-02-16,2011-08-26
english,1,1,1
developer,Kisareth Studios,Irbynx,GSC Game World
publisher,Kisareth Studios,Irbynx,GSC World Publishing
platforms,windows,windows;linux,windows
required_age,0,0,0
categories,Single-player;Steam Achievements;Full controll...,Single-player;Steam Achievements;Steam Cloud;S...,Single-player
genres,Indie;RPG,Indie,Strategy


In [7]:
df["minimum_owners"] = df ["owners"].apply(lambda v : int(v.split("-")[0]))
df = df.sort_values("minimum_owners",ascending = False)

In [8]:
from IPython.display import display , HTML
def display_game(row):
    html =''
    html +=  f'<h3>{row["name"]}</h3>'
    html +=  f'<strong>{row["genres"]}</strong>'
    html +=  f'<p>{row["short_description"]}</p>'
    html +=  f'<img src ="{row["header_image"]}">'
    display(HTML(html))
    
for _, row in df.sample(1, random_state=0).iterrows():
    display_game(row)

In [12]:
%%time
import numpy as np
top_df = df.sort_values("minimum_owners",ascending = False).head(4000)
embeddings = []
batches = np.array_split(top_df["text"], len(top_df) // 50)
for idx, chunk_text in enumerate(batches):
    if idx % 5 == 0:
        print(f"{idx} / {len(batches)}")
    embeddings +=  list(angle.encode(list(chunk_text), to_numpy=True))
top_df["embedding"] = embeddings

0 / 80
5 / 80
10 / 80
15 / 80
20 / 80
25 / 80
30 / 80
35 / 80
40 / 80
45 / 80
50 / 80
55 / 80
60 / 80
65 / 80
70 / 80
75 / 80
CPU times: total: 2h 18min 28s
Wall time: 27min 4s


In [4]:
from sklearn.metrics.pairwise import cosine_distances
game_name = 'Factorio'
game_row = top_df[top_df["name"] == game_name].iloc[0]
distances = cosine_distances(np.array([game_row.embedding]), np.array(top_df.embedding.tolist()))[0]
sorted_indices = distances.argsort()
for idx in sorted_indices[:8]:
    similar_game = top_df.iloc[idx]
    display_game(similar_game)

In [18]:
game_texts = []
for _, row in df.iterrows():
    game_text = f'''Game name : {row["name"]}
Developer : {row["developer"]}
Genres : {row["genres"]}
Summary : {row["short_description"]}'''
    game_texts.append(game_text)
df["text"] = game_texts

In [19]:
top_df.to_parquet("data/game_database.parquet")

In [ ]:
import textdistance

In [1]:
textdistance.levenshtein("path of ele","path of exile")

NameError: name 'textdistance' is not defined

In [ ]:
top_df["name"].apply(lambda v : textdistance.levenshtein(v, "path of exale")).sort_values()

In [3]:
top_df.loc[1772]

appid                                                              238960
name                                                        Path of Exile
release_date                                                   2013-10-23
english                                                                 1
developer                                             Grinding Gear Games
publisher                                             Grinding Gear Games
platforms                                                         windows
required_age                                                            0
categories              Single-player;Multi-player;Online Multi-Player...
genres                  Action;Adventure;Free to Play;Indie;Massively ...
steamspy_tags                      Free to Play;Action RPG;Hack and Slash
achievements                                                          120
positive_ratings                                                    71593
negative_ratings                      

In [ ]:
print(top_df["name"].head(40).tolist())

In [ ]:
df["embedding"] = list(angle.encode(df["text"].tolist(), to_numpy=True))

In [ ]:
list(angle.encode(df["short_description"].tolist()[:100], to_numpy=True))

In [ ]:
distances[distances.argsort()]

In [ ]:
#pip install -U angle-emb

In [ ]:
#pip install tf-keras

In [ ]:
vec = angle.encode('hello world', to_numpy = True)
print(vec)
vecs=angle.encode(['hello world1','hello world2'], to_numpy=True)
print(vecs)

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(angle.encode(['beef stakes is delicious ',
                              'the capital of Morocco is Rabat',
                              'I love meat']))[0]

In [20]:
import dill

with open('all_variables.dill', 'wb') as f:
    dill.dump_session(f)


In [2]:
import dill
with open('all_variables.dill', 'rb') as f:
    dill.load_session(f)


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
